In [ ]:
# Paso 2: Importar librerías
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch
from transformers import EarlyStoppingCallback

# Paso 3: Cargar y preparar el dataset
df = pd.read_csv("../data/mails_dataset.csv")  

# Concatenar asunto y cuerpo separados por espacio
df['text_combined'] = df['subject'].fillna('') + ' </s> ' + df['text'].fillna('')

# Nos centramos en la columna combinada y la etiqueta sentiment
df = df[['text_combined', 'sentiment']].dropna()

# Codificar etiquetas
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["sentiment"])

# Guardar las etiquetas para decodificar luego
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}
print(label_encoder.classes_)

# Paso 4: Crear Dataset de Hugging Face
dataset = Dataset.from_pandas(df.rename(columns={"text_combined": "text", "label": "label"}))

# Paso 5: Tokenización
model_name = "pysentimiento/robertuito-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

dataset = dataset.map(tokenize, batched=True)

# División entrenamiento y validación
dataset = dataset.train_test_split(test_size=0.2)

# Paso 6: Cargar modelo preentrenado
num_labels = len(label2id)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# Paso 7: Configurar entrenamiento
training_args = TrainingArguments(
    output_dir="./sentiment_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,      
    metric_for_best_model="f1",        
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_dir="./logs_sentiment",
    logging_steps=10
)


# Paso 8: Función de evaluación
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

# Paso 9: Entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Paso 10: Entrenar
trainer.train()

# Paso 11: Guardar el modelo y tokenizer
trainer.save_model("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")


['negativo' 'neutro' 'positivo']


c:\Users\Usuario\Desktop\MailClassifier\.venv\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/222 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pysentimiento/robertuito-base-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/138 [00:00<?, ?it/s]

...

c:\Users\Usuario\Desktop\MailClassifier\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 0.067, 'grad_norm': 0.8798983097076416, 'learning_rate': 2.6086956521739132e-06, 'epoch': 5.22}
{'loss': 0.03, 'grad_norm': 1.262814998626709, 'learning_rate': 1.1594202898550726e-06, 'epoch': 5.65}


  0%|          | 0/6 [00:00<?, ?it/s]

{'eval_loss': 0.24490036070346832, 'eval_accuracy': 0.9333333333333333, 'eval_f1': 0.9320158102766799, 'eval_runtime': 1.1993, 'eval_samples_per_second': 37.521, 'eval_steps_per_second': 5.003, 'epoch': 6.0}
{'train_runtime': 144.533, 'train_samples_per_second': 7.348, 'train_steps_per_second': 0.955, 'train_loss': 0.29218123032562976, 'epoch': 6.0}


('./sentiment_model\\tokenizer_config.json',
 './sentiment_model\\special_tokens_map.json',
 './sentiment_model\\tokenizer.json')

In [2]:
from sklearn.metrics import classification_report
import numpy as np

# Obtener predicciones sobre el set de evaluación
predictions = trainer.predict(dataset["test"])
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

# Mostrar el reporte con los nombres reales de las clases
print("Reporte de clasificación por clase:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))


c:\Users\Usuario\Desktop\MailClassifier\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


  0%|          | 0/6 [00:00<?, ?it/s]

Reporte de clasificación por clase:
              precision    recall  f1-score   support

    negativo       1.00      1.00      1.00        17
      neutro       1.00      0.77      0.87        13
    positivo       0.83      1.00      0.91        15

    accuracy                           0.93        45
   macro avg       0.94      0.92      0.93        45
weighted avg       0.94      0.93      0.93        45



In [9]:
from transformers import pipeline
import pandas as pd

# Carga el pipeline con tu modelo local
sentiment_classifier = pipeline(
    "text-classification",
    model="./sentiment_model",
    tokenizer="./sentiment_model"
    # No usar return_all_scores para obtener solo la mejor predicción
)

# Ejemplos de correos reales (asunto + cuerpo)
test_emails = [
    """Asunto: Pedido recibido con retraso
    Hola equipo,
    Quería informar que mi pedido llegó con dos días de retraso, aunque el producto está en buen estado.
    Gracias.""",
    """Asunto: Producto defectuoso
    Estimados,
    Recibí el producto con un defecto y quisiera solicitar un reemplazo urgente.
    Espero una pronta respuesta.""",
    """Asunto: Excelente servicio
    Buenas tardes,
    Solo quería agradecer la rapidez y eficiencia en la entrega de mi pedido. Muy satisfecho con el servicio.""",
    """Asunto: Problemas con la factura
    Hola,
    He detectado cargos que no reconozco en la factura y necesito que se revise cuanto antes.
    Saludos.""",
    """Asunto: Consulta sobre producto
    Buen día,
    Me gustaría saber si el modelo X200 está disponible en color negro y si tiene garantía extendida.
    Gracias.""",
    """Asunto: Gracias por la ayuda
    Quiero expresar mi agradecimiento por la asistencia que me brindaron ayer. Todo fue excelente.""",
    """Asunto: Pedido incompleto
    Buenos días,
    El pedido que recibí está incompleto. Faltan varios artículos que ya he pagado.
    Espero una solución inmediata.""",
    """Asunto: Solicitud de información
    Estimados,
    Podrían enviarme los detalles técnicos del producto Z45, por favor?
    Quedo atento a su respuesta.""",
    """Asunto: Felicitaciones por el servicio
    Buen trabajo con el soporte técnico, me ayudaron rápido y resolvieron todo a la perfección.""",
    """Asunto: Incidencia no resuelta
    Llevo días esperando que resuelvan el problema con mi pedido, pero no recibo respuestas claras.
    Esto es muy frustrante."""
]

# Etiquetas según entrenamiento (ajusta si es necesario)
label_map = {0: "negativo", 1: "neutro", 2: "positivo"}

# Clasificar cada correo y mostrar resultado
for email_text in test_emails:
    output = sentiment_classifier(email_text)[0]  # output es un dict con keys 'label' y 'score'
    label = output['label'].lower()
    score = output['score']
    print(f"Texto: {email_text}\nPredicción: {label}, Confianza: {score:.2f}\n")


Texto: Asunto: Pedido recibido con retraso
    Hola equipo,
    Quería informar que mi pedido llegó con dos días de retraso, aunque el producto está en buen estado.
    Gracias.
Predicción: neutro, Confianza: 0.53

Texto: Asunto: Producto defectuoso
    Estimados,
    Recibí el producto con un defecto y quisiera solicitar un reemplazo urgente.
    Espero una pronta respuesta.
Predicción: negativo, Confianza: 0.47

Texto: Asunto: Excelente servicio
    Buenas tardes,
    Solo quería agradecer la rapidez y eficiencia en la entrega de mi pedido. Muy satisfecho con el servicio.
Predicción: positivo, Confianza: 0.75

Texto: Asunto: Problemas con la factura
    Hola,
    He detectado cargos que no reconozco en la factura y necesito que se revise cuanto antes.
    Saludos.
Predicción: negativo, Confianza: 0.49

Texto: Asunto: Consulta sobre producto
    Buen día,
    Me gustaría saber si el modelo X200 está disponible en color negro y si tiene garantía extendida.
    Gracias.
Predicción: neut